In [3]:
# !pip install dspy-ai

# introduction


# load dependencies


In [6]:
import os

import dspy
from dotenv import find_dotenv, load_dotenv
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric

load_dotenv(find_dotenv("../../.env"))


model_id = "llama3-70b-8192"

# set up the LM


In [7]:
llm = dspy.GROQ(model=model_id, max_tokens=250, api_key=os.environ["GROQ_API_KEY"])

dspy.settings.configure(lm=llm)

# load dataset


In [15]:
# load math questions from the GSM8K dataset

gsm8k = GSM8K()
gsm8k_trainset, gsm8k_devset = gsm8k.train[:10], gsm8k.dev[:10]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

100%|██████████| 1319/1319 [00:00<00:00, 59941.35it/s]


In [21]:
print(gsm8k_trainset[0])

Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 'gold_reasoning': "Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}) (input_keys={'question'})


# define the module


In [24]:
# define a custom program that utilizes the `ChainOfThought` module
# to perform step by step reasoning to generate answers:
# class ChainOfThought(Predict):
#     def __init__(self, signature, rationale_type=None, activated=True, **config):
#         super().__init__(signature, **config)

#         self.activated = activated

#         signature = ensure_signature(self.signature)
#         *_keys, last_key = signature.output_fields.keys()

#         rationale_type = rationale_type or dspy.OutputField(
#             prefix="Reasoning: Let's think step by step in order to",
#             desc="${produce the " + last_key + "}. We ...",
#         )

#         self.extended_signature = signature.prepend("rationale", rationale_type, type_=str)


class CoT(dspy.Module):
    def __init__(self):
        self.prog = dspy.ChainOfThought("question -> answer")

    def forward(self, question):
        return self.prog(question=question)

# compile and evaluate the model


In [25]:
from dspy.teleprompt import BootstrapFewShot

# set up the optimizer
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)

# optimizer
# compiles the BootstrapFewShot instance by performing bootstrapping
# to refine the student predictor
teleprompter = BootstrapFewShot(metric=gsm8k_metric, **config)
optimized_cot = teleprompter.compile(CoT(), trainset=gsm8k_trainset)

 50%|█████     | 5/10 [00:04<00:04,  1.14it/s]

Bootstrapped 4 full traces after 6 examples in round 0.


# evaluate


In [27]:
from dspy.evaluate import Evaluate

# set up the evaluator
evaluate = Evaluate(
    devset=gsm8k_devset,
    metric=gsm8k_metric,
    num_threads=4,
    display_progress=True,
    display_table=0,
)

# evaluate the optimized_cot program
evaluate(optimized_cot)

Average Metric: 10 / 10  (100.0): 100%|██████████| 10/10 [01:02<00:00,  6.26s/it]


100.0

# inspect the model's history


In [28]:
# review the most recent generations through inspecting the model's history

llm.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?
Reasoning: Let's think step by step in order to Here is the completed solution: Question: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score? Reasoning: Let's think step by step in order to find Marion's score. We know Ella got 4 incorrect answers, so she got 40 - 4 = 36 correct answers. Marion got 6 more than half the score of Ella, so Marion got (36 / 2) + 6 = 24 correct answers.
Answer: 24

---

Question: Stephen made 10 round trips up 

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?\nReasoning: Let's think step by step in order to Here is the completed solution: Question: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score? Reasoning: Let's think step by step in order to find Marion's score. We know Ella got 4 incorrect answers, so she got 40 - 4 = 36 correct answers. Marion got 6 more than half the score of Ella, so Marion got (36 / 2) + 6 = 24 correct answers.\nAnswer: 24\n\n---\n\nQuestion: Stephen m

# references

- [minimal working example](https://dspy-docs.vercel.app/docs/quick-start/minimal-example)
